[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1bKD5e3XQuu05b7YGCMCIpFmjFwjzC3ar?usp=sharing)

In [ ]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import cv2
import os
from google.colab import drive
drive.mount('/content/gdrive')

%matplotlib inline

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
PATH_OF_DATA= '/content/gdrive/"My Drive"/final_fg_data/'
!ls {PATH_OF_DATA}

Test  Train


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
num_epochs = 10
batch_size = 30
num_classes = 11
learning_rate = 0.0001
img_rows = 64
img_cols = 64

TRANSFORM_IMG = transforms.Compose([
    transforms.Resize((img_rows,img_cols)),
    transforms.CenterCrop((img_rows,img_cols)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
    ])

TRAIN_DIR = '/content/gdrive/My Drive/final_fg_data/Train'
TEST_DIR = '/content/gdrive/My Drive/final_fg_data/Test'


# for country in os.listdir(DATA_DIR):
#     for each_flag in os.listdir(DATA_DIR+'/'+country): 
#         actual_path = os.path.join(DATA_DIR,country,each_flag)
#         img_data = cv2.imread(actual_path)
#         img_data = cv2.resize(img_data, (img_rows, img_cols))

train_data = torchvision.datasets.ImageFolder(root= TRAIN_DIR, transform=TRANSFORM_IMG)
test_data = torchvision.datasets.ImageFolder(root= TEST_DIR, transform = TRANSFORM_IMG)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True,  num_workers=4)
test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = batch_size, shuffle = True)

In [ ]:
class Conv(nn.Module):
    def __init__(self, num_classes):
        super(Conv, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,16, kernel_size=5, stride = 1, padding = 2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride=2),
#             self.drop_layer = nn.Dropout(p=p)
            )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32,kernel_size = 5, padding =2, stride =1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride =2),
#             self.drop_layer = nn.Dropout(p=p)
            )
        self.fc = nn.Linear(16*16*32, num_classes)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
#         print(out.size(1))   #this is to check your implementation
#         print(out.size(2))
#         print(out.size(3))
        out = out.reshape(out.size(0), -1)#here we provide a batch of image so out.size(0) is the batch_size
        out = self.fc(out)
        return out

In [ ]:
model = Conv(num_classes,).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)
# out = model(train_dataset[0][0].reshape(1,1,28,28))#1-no. of images, 1-depth of the image, height, width

# For updating learning rate
# def update_lr(optimizer, lr):    
#     for param_group in optimizer.param_groups:
#         param_group['lr'] = lr

total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
#         print(images.shape)
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
#         if((epoch+1)%20 == 0):
#             learning_rate /= 4
#             update_lr(optimizer, learning_rate)
    

model.eval()  #it is typically used for batchnorm so that it uses moving variance and mean instead of the whole batch

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Train Accuracy of the model with {} images: {} %'.format(total,100 * correct / total))
    

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Test Accuracy of the model with {} images: {} %'.format(total,100 * correct / total))
    

Epoch [1/10], Step [10/77], Loss: 2.3095
Epoch [1/10], Step [20/77], Loss: 2.1030
Epoch [1/10], Step [30/77], Loss: 2.0370
Epoch [1/10], Step [40/77], Loss: 1.8555
Epoch [1/10], Step [50/77], Loss: 1.6996
Epoch [1/10], Step [60/77], Loss: 1.3685
Epoch [1/10], Step [70/77], Loss: 1.1334
Epoch [2/10], Step [10/77], Loss: 1.0276
Epoch [2/10], Step [20/77], Loss: 1.1550
Epoch [2/10], Step [30/77], Loss: 0.9196
Epoch [2/10], Step [40/77], Loss: 0.8090
Epoch [2/10], Step [50/77], Loss: 0.8380
Epoch [2/10], Step [60/77], Loss: 0.8817
Epoch [2/10], Step [70/77], Loss: 0.7527
Epoch [3/10], Step [10/77], Loss: 0.7139
Epoch [3/10], Step [20/77], Loss: 0.7876
Epoch [3/10], Step [30/77], Loss: 0.7469
Epoch [3/10], Step [40/77], Loss: 0.5093
Epoch [3/10], Step [50/77], Loss: 0.7507
Epoch [3/10], Step [60/77], Loss: 0.7317
Epoch [3/10], Step [70/77], Loss: 0.6714
Epoch [4/10], Step [10/77], Loss: 0.6449
Epoch [4/10], Step [20/77], Loss: 0.5365
Epoch [4/10], Step [30/77], Loss: 0.7208
Epoch [4/10], St

In [ ]:
stateDict = '/content/gdrive/My Drive/final_fg_data/savedModel/stateDict.pt'
wholeModel = '/content/gdrive/My Drive/final_fg_data/savedModel/wholeModel.pt'
torch.save(model.state_dict(), stateDict)
torch.save(model, wholeModel)

# Later to restore write:
# 1. for using statedict:
#   model.load_state_dict(torch.load(stateDict))
#   model.eval()
# 2. for using whole Model:
#   model = torch.load(wholeModel)
# for more details of using the trained model refer to : https://stackoverflow.com/questions/42703500/best-way-to-save-a-trained-model-in-pytorch ,
# https://pytorch.org/tutorials/beginner/saving_loading_models.html

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Conv. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
